In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/share/software/user/open/py-scipy/1.6.3_py39/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2025-11-23 14:42:53.683015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 14:42:55.869332: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-23 14:43:04.011196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.

## Scaling all sfe and climate data to be between 0 and 1
https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [2]:
# read in list of arrays for each pixel - these are fully filtered
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_filtered_window.npz", allow_pickle = True)["sfe_window"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_filtered_window.npz", allow_pickle = True)["climate_window"]



In [3]:
print(sfe_pre.shape)
print(climate_pre.shape)

(3799134,)
(3799134, 6, 8)


In [4]:
sfe_pre_rs = sfe_pre.reshape(-1,1) # single feature
sfe_pre_rs.shape


(3799134, 1)

In [5]:
# climate data is 3d, needs to be 2d for minmaxscaler
# so reshape and then turn back to original shape after processing
climate_pre_ogshape = climate_pre.shape
print(climate_pre_ogshape)

climate_pre_rs = climate_pre.reshape(-1, climate_pre_ogshape[2]) # (pixel-month pairs * timesteps, 8)
climate_pre_rs.shape

(3799134, 6, 8)


(22794804, 8)

In [6]:
scaler_sfe = MinMaxScaler(feature_range=(0,1))
scaler_climate = MinMaxScaler(feature_range=(0,1))

sfe_pre_scaled = scaler_sfe.fit_transform(sfe_pre_rs)
climate_pre_scaled = scaler_climate.fit_transform(climate_pre_rs)



In [7]:
print(sfe_pre_scaled.min(), sfe_pre_scaled.max())
print(climate_pre_scaled.min(), climate_pre_scaled.max())


0.0 1.0
0.0 1.0000000000000002


In [8]:
sfe_back = scaler_sfe.inverse_transform(sfe_pre_scaled)

sfe_back.shape
print(sfe_back.min(), sfe_back.max())
print(sfe_pre_rs.min(), sfe_pre_rs.max())


-0.4033722536155057 5.223653647385592
-0.40337225361550577 5.223653647385593


In [9]:
# change climate data back to orig shape
climate_pre_scaled = climate_pre_scaled.reshape(climate_pre_ogshape)

In [10]:
climate_pre_scaled = np.array(climate_pre_scaled)
sfe_pre_scaled = np.array(sfe_pre_scaled)

# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_pre_scaled, dtype=object)
climate_obj = np.array(climate_pre_scaled, dtype=object)

In [11]:
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_window_scaled.npz", sfe_pre = sfe_obj)
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_window_scaled.npz", climate_pre =climate_obj)

## training the model

https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/

In [12]:
# reading in sliding window data, was also scaled
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_window_scaled.npz", allow_pickle = True)["sfe_pre"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_window_scaled.npz", allow_pickle = True)["climate_pre"]

In [13]:
# converting from objects
sfe_pre = np.array(sfe_pre, dtype=np.float32)
climate_pre = np.array(climate_pre, dtype=np.float32)


In [14]:
climate_train, climate_test, sfe_train, sfe_test = train_test_split(
    climate_pre, sfe_pre, test_size=0.1, shuffle=False
)

In [15]:
model = Sequential()
model.add(BatchNormalization(input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # input shape is the length of the sequence, each input feature
model.add(LSTM(units=64, return_sequences=False,
          input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # return one output per sequence
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics = ['mae'])
model.summary()

2025-11-23 14:45:59.487153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 14:46:05.414762: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38199 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 6, 8)             32        
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 18,785
Trainable params: 18,769
Non-trainable params: 16
_________________________________________________________________


In [16]:
history = model.fit(climate_train, sfe_train, epochs=20, batch_size=64, validation_split=0.1)

predictions = model.predict(climate_test)
np.save("/scratch/users/ashdef/model_out/baseline/predictions_scaled.npy", predictions)

predictions_original = scaler_sfe.inverse_transform(predictions)
np.save("/scratch/users/ashdef/model_out/baseline/predictions_original.npy", predictions_original)

np.save("/scratch/users/ashdef/model_out/baseline/sfe_test_scaled.npy", sfe_test)
    
sfe_test_original = scaler_sfe.inverse_transform(sfe_test)
np.save("/scratch/users/ashdef/model_out/baseline/sfe_test_original.npy", sfe_test_original)

model.save("/scratch/users/ashdef/model_out/baseline/baseline.keras")

rmse = mean_squared_error(sfe_test_original, predictions_original, squared=False)
mae = mean_absolute_error(sfe_test_original, predictions_original)
r2 = r2_score(sfe_test_original, predictions_original)

print(rmse, mae, r2)


Epoch 1/20


2025-11-23 14:46:34.801089: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2025-11-23 14:46:38.471872: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


48083/48083 [==============================] - 258s 5ms/step - loss: 0.0018 - mae: 0.0307 - val_loss: 0.0015 - val_mae: 0.0284
Epoch 2/20
48083/48083 [==============================] - 253s 5ms/step - loss: 0.0013 - mae: 0.0258 - val_loss: 0.0013 - val_mae: 0.0258
Epoch 3/20
48083/48083 [==============================] - 255s 5ms/step - loss: 0.0011 - mae: 0.0243 - val_loss: 0.0012 - val_mae: 0.0246
Epoch 4/20
48083/48083 [==============================] - 256s 5ms/step - loss: 0.0010 - mae: 0.0235 - val_loss: 0.0011 - val_mae: 0.0243
Epoch 5/20
48083/48083 [==============================] - 256s 5ms/step - loss: 0.0010 - mae: 0.0230 - val_loss: 0.0011 - val_mae: 0.0241
Epoch 6/20
48083/48083 [==============================] - 256s 5ms/step - loss: 9.7522e-04 - mae: 0.0227 - val_loss: 0.0011 - val_mae: 0.0242
Epoch 7/20
48083/48083 [==============================] - 257s 5ms/step - loss: 9.5307e-04 - mae: 0.0225 - val_loss: 0.0011 - val_mae: 0.0242
Epoch 8/20
48083/48083 [=============

In [17]:
history.history

{'loss': [0.001811954309232533,
  0.0012670101132243872,
  0.001120761618949473,
  0.0010485830716788769,
  0.0010038816835731268,
  0.0009752186015248299,
  0.0009530686656944454,
  0.0009364021243527532,
  0.0009227204136550426,
  0.0009114684071391821,
  0.0009037636918947101,
  0.0008949267212301493,
  0.0008876398205757141,
  0.0008832838502712548,
  0.0008762608049437404,
  0.000871458207257092,
  0.0008665640489198267,
  0.0008633817778900266,
  0.0008584854658693075,
  0.0008554259547963738],
 'mae': [0.030731363222002983,
  0.025826470926404,
  0.024318592622876167,
  0.023535270243883133,
  0.02303900197148323,
  0.022709941491484642,
  0.02245219796895981,
  0.022254852578043938,
  0.02208399400115013,
  0.021951431408524513,
  0.02185666374862194,
  0.02175476960837841,
  0.021658122539520264,
  0.021600965410470963,
  0.0215213131159544,
  0.02147303894162178,
  0.02140660211443901,
  0.021364590153098106,
  0.02130569890141487,
  0.02126702293753624],
 'val_loss': [0.0015

In [18]:
np.save("/scratch/users/ashdef/model_out/baseline/training_history.npy", history.history)